In [119]:
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics
from matplotlib import pyplot as plt
import time
import sys
import cv2 as cv
import multiprocessing
import glob

In [120]:
#img = cv.imread('input_images/left0307.jpg',1)
#transform(img)
def color(mat,image):
    sumred = 0
    sumgrn = 0
    sumblu = 0
    for x in range(len(mat)):
        sumred += image [mat[x][1]] [mat[x][0]] [2]
        sumgrn += image [mat[x][1]] [mat[x][0]] [1]
        sumblu += image [mat[x][1]] [mat[x][0]] [0]
    #print(sumred/len(mat))
    #print(sumgrn/len(mat))
    #print(sumblu/len(mat))
    if ((sumred/len(mat)) > 100 and (sumred/len(mat)) < 180 and (sumgrn/len(mat)) > 30 and (sumgrn/len(mat)) < 115  and (sumblu/len(mat)) > 30 and (sumblu/len(mat)) < 110):
        #print('red')
        return 'red'
    if ((sumred/len(mat)) > 45 and (sumred/len(mat)) < 100 and (sumgrn/len(mat)) > 75 and (sumgrn/len(mat)) < 180  and (sumblu/len(mat)) > 50 and (sumblu/len(mat)) < 130):
        #print('green')
        return 'green'
    else:
        #print('none')
        return 'none'
    
def rectangle(mat,color,image,original):
    rect = cv.minAreaRect(mat)
    height = rect[1][1]
    width = rect[1][0]
    x = rect[0][0] - width/2
    x = np.int0(x)
    y = rect[0][1] - height/1.7
    y = np.int0(y)
    font = cv.FONT_HERSHEY_SIMPLEX
    if(color == 'none'):
        cv.putText(original,'noise',(x,y), font, 0.5,(255,255,255),2,cv.LINE_AA)
    elif((1.5*width < height and 3.5*width > height) or (1.5*height < width and 3.5*height > width)):
        cv.putText(original,color+' buoy',(x,y), font, 0.5,(255,255,255),2,cv.LINE_AA)
    box = cv.boxPoints(rect)
    box = np.int0(box)
    cv.drawContours(original,[box],0,(255,0,255),2)
    return original

def pp(image):
    image = np.array(image[0:800][0:515])
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    image[np.logical_not(np.logical_or(np.logical_and(image[:,:,0] > 70, image[:,:,0] < 90),np.logical_or(image[:,:,0] >= 170, image[:,:,0] < 10)))] = [0,0,0]
    image = cv.cvtColor(image,cv.COLOR_HSV2BGR)
    return(image)

def transform(img):
    image = pp(img)
    img2 = dbscan(image, img)
    #img3 = plot(img2)
    return img2

def dbscan(image, original): 
    img = image.copy()
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    ret,thresh1 = cv.threshold(img,40,255,cv.THRESH_BINARY)

    nonzero = np.nonzero(thresh1)

    yp = np.array(nonzero[0])
    xp = np.array(nonzero[1])


    X=np.column_stack((xp,yp))
    if (len(X) > 0):
        db = DBSCAN(eps=3, min_samples=20).fit(X)
        core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
        core_samples_mask[db.core_sample_indices_] = True
        labels = db.labels_
        # Number of clusters in labels, ignoring noise if present.
        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise_ = list(labels).count(-1)


        array = zip(xp,yp,labels)
        sort = sorted(list(array), key=lambda x: x[2])
        x = 0

        while (len(sort) > 0 and sort[x][2] == -1):
            del sort[x]

        if (len(sort) > 0):
            unique_labels = sorted(set(labels))
            if (unique_labels[0] == -1):
                unique_labels.remove(-1)
            points = sort.copy()
            points = np.delete(points,2,1)

            i = 0
            x = 0
            l = []

            while (x < len(unique_labels)):
                if (i < len(sort) and sort[i][2] == x):
                    l.append(points[i])
                    i+=1
                else:
                    x+=1
                    clr = color(np.array(l),image)
                    tb = rectangle(np.array(l),clr,image,original)
                    l = []
            original = tb   
    return original

In [121]:
def read(x):
    img2 = cv.imread('images/left' + str(x).zfill(4) + '.jpg',1)
    result1 = transform(img2)
    cv.imwrite('output_images/test1/test' + str(x).zfill(4) + '.jpg', result1)

In [122]:
read(x)

In [123]:

if __name__=='__main__':
    starttime = time.time()
    pool = multiprocessing.Pool(multiprocessing.cpu_count())
    
    
    pool.map(read,range(2927),30)
    pool.close()
    pool.join()
    
    print('That took {} seconds',format(time.time() - starttime))


That took {} seconds 100.02417612075806


In [124]:
img_arr = []
starttime = time.time()
for x in range(0,2927):
    img = cv.imread('output_images/test1/test' + str(x).zfill(4) + '.jpg',1)
    height, width, layers = img.shape
    size = (width,height)
    img_arr.append(img)
out = cv.VideoWriter('output_images/alice_images.avi',cv.VideoWriter_fourcc(*'DIVX'), 20, size)
for i in range(len(img_arr)):
    out.write(img_arr[i])
out.release()